In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='6ce270de-788d-4e07-8bae-3206860a6387'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='ajinda10_0051550143'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/keys/jiabhi_bastion'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_NAME']=os.environ['HOME']+'/work/keys/jiabhi_sliver'
os.environ['FABRIC_SLICE_PUBLIC_KEY_NAME']=os.environ['HOME']+'/work/keys/jiabhi_sliver.pub'
os.environ['FABRIC_CONFIG_LOCATION']=os.environ['HOME']+'/work/keys/fabric-ssh-config'
# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

In [2]:
FABRIC_BASTION_USERNAME = os.environ['FABRIC_BASTION_USERNAME']
FABRIC_BASTION_KEY_LOCATION = os.environ['FABRIC_BASTION_KEY_LOCATION']
FABRIC_SLICE_PRIVATE_KEY_FILE = os.environ['FABRIC_SLICE_PRIVATE_KEY_NAME']
FABRIC_BASTION_HOST = os.environ['FABRIC_BASTION_HOST']
FABRIC_CONFIG_LOCATION = os.environ['FABRIC_CONFIG_LOCATION']

In [3]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed.slice_editor import Capacities

In [4]:
manager = fablib.get_default_fablib_manager()
try:
    print(f"{manager}")
except Exception as e:
    print(f"Exception: {e}")

In [5]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'complex_tcp_ecn_slice1'

[site1,site2,site3] = ['UTAH', 'UTAH', 'UTAH']
# fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"
h11_name = "h11"
h22_name = "h22"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h11_subnet=IPv4Network('10.0.1.0/24')
h11_addr=IPv4Address('10.0.1.11')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

h22_subnet=IPv4Network('10.0.2.0/24')
h22_addr=IPv4Address('10.0.2.22')

h3_subnet=IPv4Network('10.0.3.0/24')
h3_addr=IPv4Address('10.0.3.3')

host_cores = 2
host_ram = 8
host_disk = 10

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_h11_name = 'net_h11'
net_h22_name = 'net_h22'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
image = 'default_ubuntu_20'

Sites: UTAH,UTAH,UTAH


In [6]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_local = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()[0]
    s11_iface_local = s1.add_component(model='NIC_Basic', name="s11_local_nic").get_interfaces()[0]
    s1_iface_to_s2 = s1.add_component(model='NIC_Basic', name="s1_switch_nic1").get_interfaces()[0]
    s1_iface_to_s3 = s1.add_component(model='NIC_Basic', name="s1_switch_nic2").get_interfaces()[0]

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site2,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2_iface_local = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()[0]
    s22_iface_local = s2.add_component(model='NIC_Basic', name="s22_local_nic").get_interfaces()[0]
    s2_iface_to_s1 = s2.add_component(model='NIC_Basic', name="s2_switch_nic1").get_interfaces()[0]
    s2_iface_to_s3 = s2.add_component(model='NIC_Basic', name="s2_switch_nic2").get_interfaces()[0]
    
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site3,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3_iface_local = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()[0]
    s3_iface_to_s1 = s3.add_component(model='NIC_Basic', name="s3_switch_nic1").get_interfaces()[0]
    s3_iface_to_s2 = s3.add_component(model='NIC_Basic', name="s3_switch_nic2").get_interfaces()[0]    
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h11
    h11 = slice.add_node(name=h11_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h11_iface = h11.add_component(model='NIC_Basic', name="h11_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    # Add host node h22
    h22 = slice.add_node(name=h22_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h22_iface = h22.add_component(model='NIC_Basic', name="h22_nic").get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net11 = slice.add_l2network(name=net_h11_name, interfaces=[s11_iface_local, h11_iface])

    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net22 = slice.add_l2network(name=net_h22_name, interfaces=[s22_iface_local, h22_iface])
    
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


-----------  ------------------------------------
Slice Name   complex_tcp_ecn_slice1
Slice ID     c402a083-87bc-426c-ad56-66228e8e1b8b
Slice State  StableOK
Lease End    2022-11-21 21:14:47 +0000
-----------  ------------------------------------

Retry: 15, Time: 290 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                        State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  -----------------------------------  -------  -------
de479732-c87f-413b-b139-10d6fa675ec3  s1      UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fecd:481d  Active
576d5359-944f-4a64-b13c-334aa94dfa2d  s2      UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fe23:2cd3  Active
2f96cf52-6897-4408-8f2a-292bdd8

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ID                 de479732-c87f-413b-b139-10d6fa675ec3
Name               s1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               utah-w5.fabric-testbed.net
Site               UTAH
Management IP      2001:1948:417:7:f816:3eff:fecd:481d
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fecd:481d
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
-----------------  -----------------------------------------------------------------------------------------------------------------------------------------

In [8]:
# variables specific to this slice
h1_ip = str(slice.get_node(h1_name).get_management_ip())
h1_user =  str(slice.get_node(h1_name).get_username())
h1_iface = slice.get_node(h1_name).get_interfaces()[0].get_os_interface()

h11_ip = str(slice.get_node(h11_name).get_management_ip())
h11_user =  str(slice.get_node(h11_name).get_username())
h11_iface = slice.get_node(h11_name).get_interfaces()[0].get_os_interface()

h2_ip = str(slice.get_node(h2_name).get_management_ip())
h2_user =  str(slice.get_node(h2_name).get_username())
h2_iface = slice.get_node(h2_name).get_interfaces()[0].get_os_interface()

h22_ip = str(slice.get_node(h22_name).get_management_ip())
h22_user =  str(slice.get_node(h22_name).get_username())
h22_iface = slice.get_node(h22_name).get_interfaces()[0].get_os_interface()

s1_ip = str(slice.get_node(s1_name).get_management_ip())
s1_user =  str(slice.get_node(s1_name).get_username())
s1_iface_h1 = slice.get_node(s1_name).get_component("s1_local_nic").get_interfaces()[0].get_os_interface()
s1_iface_h11 = slice.get_node(s1_name).get_component("s11_local_nic").get_interfaces()[0].get_os_interface()

s1_iface_s2 = slice.get_node(s1_name).get_component("s1_switch_nic1").get_interfaces()[0].get_os_interface()
s2_iface_s1 = slice.get_node(s2_name).get_component("s2_switch_nic1").get_interfaces()[0].get_os_interface()

s2_ip = str(slice.get_node(s2_name).get_management_ip())
s2_user =  str(slice.get_node(s2_name).get_username())
s2_iface_h2 = slice.get_node(s2_name).get_component("s2_local_nic").get_interfaces()[0].get_os_interface()
s2_iface_h22 = slice.get_node(s2_name).get_component("s22_local_nic").get_interfaces()[0].get_os_interface()

print(s1_iface_h1)
print(s1_iface_h11)
print(s1_iface_s2)
print(s2_iface_s1)


print(s2_iface_h2)
print(s2_iface_h22)

ens7
ens10
ens8
ens8
ens9
ens7


In [10]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$h1_user" "$h1_ip" "$h1_iface"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.1.1/24 dev $5
sudo ip route add 10.0.2.0/24 via 10.0.1.2 dev $5
sudo ip route add 10.0.3.0/24 via 10.0.1.2 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:22:48 UTC 2022

  System load:           0.0
  Usage of /:            13.8% of 9.52GB
  Memory usage:          2%
  Swap usage:            0%
  Processes:             128
  Users logged in:       0
  IPv4 address for ens3: 10.20.4.18
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fed9:85cb


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 3a:ca:ea:01:a6:be brd ff:ff:ff:ff:ff:ff
    inet 10.0.1.1/24 s

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [20]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$h11_user" "$h11_ip" "$h11_iface"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.1.11/24 dev $5
sudo ip route add 10.0.3.20/31 via 10.0.1.12 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:30:35 UTC 2022

  System load:           0.0
  Usage of /:            13.8% of 9.52GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             130
  Users logged in:       1
  IPv4 address for ens3: 10.20.4.207
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fef6:ea55
  IPv4 address for ens7: 10.0.1.11
  IPv4 address for ens7: 10.0.1.11


0 updates can be applied immediately.

New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 3e:7d:05:bc:dc:4d brd ff:ff:ff:ff:ff:ff
    inet 10.0.1.11/31 scope global ens7
       valid_lft forever preferred_lft forever
    inet 10.0.1.11/24 sc

Pseudo-terminal will not be allocated because stdin is not a terminal.
RTNETLINK answers: File exists


In [12]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$h2_user" "$h2_ip" "$h2_iface"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.3.2/24 dev $5
sudo ip route add 10.0.2.0/24 via 10.0.3.1 dev $5
sudo ip route add 10.0.1.0/24 via 10.0.3.1 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:23:12 UTC 2022

  System load:           0.0
  Usage of /:            13.8% of 9.52GB
  Memory usage:          2%
  Swap usage:            0%
  Processes:             128
  Users logged in:       0
  IPv4 address for ens3: 10.20.4.191
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fee0:7091


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 46:9d:05:77:21:0a brd ff:ff:ff:ff:ff:ff
    inet 10.0.3.2/24 

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [13]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$h22_user" "$h22_ip" "$h22_iface"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.3.22/31 dev $5
sudo ip route add 10.0.1.10/31 via 10.0.3.21 dev $5

ip addr show dev $5
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:23:57 UTC 2022

  System load:           0.0
  Usage of /:            13.8% of 9.52GB
  Memory usage:          2%
  Swap usage:            0%
  Processes:             128
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.103
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fe9a:457c


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 4a:41:0d:7b:3e:53 brd ff:ff:ff:ff:ff:ff
    inet 10.0.3.22/31

Pseudo-terminal will not be allocated because stdin is not a terminal.
Error: Nexthop has invalid gateway.


In [14]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s1_user" "$s1_ip" "$s1_iface_h1" "$s1_iface_s2"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.1.2/24 dev $5
sudo ip route add 10.0.1.0/24 via 10.0.1.1 dev $5

sudo ip addr add 10.0.2.2/24 dev $6
sudo ip route add 10.0.2.0/24 via 10.0.2.1 dev $6
sudo ip route add 10.0.3.0/24 via 10.0.2.1 dev $6
sudo sysctl -w net.ipv4.ip_forward=1

ip addr show dev $5
ip addr show dev $6
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:24:15 UTC 2022

  System load:           0.0
  Usage of /:            1.4% of 96.75GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             156
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.69
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fecd:481d


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


net.ipv4.ip_forward = 1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 32:d7:d9:6b:26:45 brd ff:ff:ff:ff:ff:f

Pseudo-terminal will not be allocated because stdin is not a terminal.
RTNETLINK answers: File exists
RTNETLINK answers: File exists


In [15]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s1_user" "$s1_ip" "$s1_iface_h11" "$s1_iface_s2"
echo $1
echo $2
echo $3
echo $4
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.1.12/31 dev $5
sudo ip route add 10.0.1.10/31 via 10.0.1.11 dev $5
sudo sysctl -w net.ipv4.ip_forward=1

ip addr show dev $5
ip addr show dev $6
ip route show

##############################################
exit
EOF

/home/fabric/work/keys/fabric-ssh-config
/home/fabric/work/keys/jiabhi_sliver
ubuntu
2001:1948:417:7:f816:3eff:fecd:481d
Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:24:45 UTC 2022

  System load:           0.08
  Usage of /:            1.4% of 96.75GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             156
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.69
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fecd:481d
  IPv4 address for ens7: 10.0.1.2
  IPv4 address for ens8: 10.0.2.2


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' t

Pseudo-terminal will not be allocated because stdin is not a terminal.
Error: Nexthop has invalid gateway.


In [16]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s2_user" "$s2_ip" "$s2_iface_s1" "$s2_iface_h2"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo ip addr add 10.0.2.1/24 dev $5
sudo ip route add 10.0.1.0/24 via 10.0.2.2 dev $5
sudo ip route add 10.0.2.0/24 via 10.0.2.2 dev $5

sudo ip addr add 10.0.3.1/24 dev $6
sudo ip route add 10.0.3.0/24 via 10.0.3.2 dev $6

sudo sysctl -w net.ipv4.ip_forward=1

ip addr show dev $5
ip addr show dev $6
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:25:08 UTC 2022

  System load:           0.06
  Usage of /:            1.4% of 96.75GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             152
  Users logged in:       0
  IPv4 address for ens3: 10.20.4.196
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fe23:2cd3


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


net.ipv4.ip_forward = 1
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2a:37:69:25:61:58 brd ff:ff:ff:ff:ff

Pseudo-terminal will not be allocated because stdin is not a terminal.
RTNETLINK answers: File exists
RTNETLINK answers: File exists


In [17]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s2_user" "$s2_ip" "$s2_iface_s1" "$s2_iface_h22"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################
sudo ip addr add 10.0.3.21/31 dev $6
sudo ip route add 10.0.3.20/31 via 10.0.3.22 dev $6

sudo sysctl -w net.ipv4.ip_forward=1

ip addr show dev $5
ip addr show dev $6
ip route show

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Sun Nov 20 21:25:37 UTC 2022

  System load:           0.03
  Usage of /:            1.4% of 96.75GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             151
  Users logged in:       0
  IPv4 address for ens3: 10.20.4.196
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fe23:2cd3
  IPv4 address for ens8: 10.0.2.1
  IPv4 address for ens9: 10.0.3.1


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.


net.ipv4.ip_forward = 1
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group d

Pseudo-terminal will not be allocated because stdin is not a terminal.
Error: Nexthop has invalid gateway.


In [12]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s2_user" "$s2_ip" "$s2_iface_s1" "$s2_iface_h2"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo tc qdisc del dev $5 root
sudo tc qdisc add dev $5 root handle 1: htb default 3
sudo tc class add dev $5 parent 1: classid 1:3 htb rate 0.5Mbit
sudo tc qdisc add dev $5 parent 1:3 handle 3: bfifo limit 0.1MB

##############################################
exit
EOF

Welcome to Ubuntu 20.04.3 LTS (GNU/Linux 5.4.0-97-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Nov 14 01:08:08 UTC 2022

  System load:           0.08
  Usage of /:            13.8% of 9.52GB
  Memory usage:          3%
  Swap usage:            0%
  Processes:             142
  Users logged in:       0
  IPv4 address for ens3: 10.20.5.30
  IPv6 address for ens3: 2001:1948:417:7:f816:3eff:fe15:8cf3
  IPv4 address for ens7: 192.168.0.1
  IPv4 address for ens8: 192.168.1.1


1 update can be applied immediately.
To see these additional updates run: apt list --upgradable


The list of available updates is more than a week old.
To check for new updates run: sudo apt update
New release '22.04.1 LTS' available.
Run 'do-release-upgrade' to upgrade to it.




Pseudo-terminal will not be allocated because stdin is not a terminal.
Error: Cannot delete qdisc with handle of zero.
Error: Cannot delete qdisc with handle of zero.


In [ ]:
%%bash -s "$FABRIC_CONFIG_LOCATION" "$FABRIC_SLICE_PRIVATE_KEY_FILE" "$s1_user" "$s1_ip" "$s1_iface_h1" "$s1_iface_s2"
ssh -F $1 -i $2 $3@$4 << EOF
##############################################

sudo tc qdisc del dev $6 root
sudo tc qdisc add dev $6 root handle 1: htb default 3
sudo tc class add dev $6 parent 1: classid 1:3 htb rate 0.5Mbit
sudo tc qdisc add dev $6 parent 1:3 handle 3: bfifo limit 0.1MB

##############################################
exit
EOF

In [13]:
node1 = slice.get_node(node1_name)
stdout, stderr = node1.execute("sudo apt-get update && sudo apt-get -y install iperf3")
print (stdout)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1822 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [301 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.2 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1293 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [183 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [628 B]
Get:12 http://security.ubuntu.com/ubuntu focal-security/univ

In [14]:
node2 = slice.get_node(node2_name)
stdout, stderr = node2.execute("sudo apt-get update && sudo apt-get -y install iperf3")
print (stdout)

Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1822 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [301 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.2 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1293 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [183 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/restrict

In [15]:
stdout, stderr = node1.execute('sudo apt -y install net-tools && sudo apt-get -y install moreutils')
print(stdout)
print(stderr)

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 179 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
Fetched 196 kB in 1s (339 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63596 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20180626.aebd88e-1ubuntu1_amd64.deb ...
Unpacking net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Setting up net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Processing triggers for man-db (2.9.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libio-pty-perl libipc-run-pe

In [16]:
stdout, stderr = node2.execute('sudo apt -y install net-tools && sudo apt-get -y install moreutils')
print(stdout)
print(stderr)

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 179 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
Fetched 196 kB in 1s (341 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63596 files and directories currently installed.)
Preparing to unpack .../net-tools_1.60+git20180626.aebd88e-1ubuntu1_amd64.deb ...
Unpacking net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Setting up net-tools (1.60+git20180626.aebd88e-1ubuntu1) ...
Processing triggers for man-db (2.9.1-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libio-pty-perl libipc-run-pe